# Enrollment Prediction - Binary Logit

### Goals

* Identify the effect of scholarship amounts on a student's decision to enroll at the UW or not
* Use the original approach of HCRC; a binary logistic regression. 

### Process

* A. Load in data and software libraries
* B. Set the option flags for this run
* C. Visualize the different factors and how they interact
* D. Normalize, clean up, and dummy-code various factors
* E. Split into training and test sets
* F. Run a binary logistic regression
* G. Evaluate the results for accuracy
* H. Save off the model for use elsewhere

### Resources

* http://blog.yhat.com/posts/logistic-regression-python-rodeo.html 
* http://machinelearningmastery.com/logistic-regression-tutorial-for-machine-learning/
* http://www.dataschool.io/logistic-regression-in-python-using-scikit-learn/
* https://www.r-bloggers.com/how-to-perform-a-logistic-regression-in-r/

## Part A: Load in data and software libraries

**Load in the software libraries**

In [1]:
# Load in software libraries
import os
import sys
import pandas as pd
import numpy as np
import pickle

# Get data-munging libraries
from sklearn import preprocessing
from sklearn.preprocessing import Binarizer, FunctionTransformer
from sklearn import decomposition

# Get ML libraries
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.kernel_approximation import Nystroem
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.pipeline import make_pipeline, make_union

import statsmodels.api as sm
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.cross_validation import train_test_split


#from tpot import TPOTClassifier

## Get propensity score libraries
from causalinference import CausalModel

# Get Viz libraries
import matplotlib.pyplot as plt
import seaborn as sns

C:\Users\laulck\AppData\Local\Continuum\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools
C:\Users\laulck\AppData\Local\Continuum\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


**Load in the data**

In [2]:
#df = pd.read_csv('Working_Data_set.csv')
#df = pd.read_csv('working_data_set_v2.csv.zip', compression='infer')
df = pd.read_csv('working_data_set_v3.csv')
retention_df = pd.read_csv('retention_info.csv')

IOError: File working_data_set_v3.csv does not exist

In [ ]:
# Drop unneeded columns
df.drop('system_key', axis=1, inplace=True)
df.drop('GenderDesc', axis=1, inplace=True)
df.drop('EthnicityDesc', axis=1, inplace=True)
df.drop('HighSchoolCode', axis=1, inplace=True)

# Drop confounding columns
df.drop('EnrollYrQtr', axis=1, inplace=True)
df.drop('FirstYrGPA', axis=1, inplace=True)
df.drop('FirstYrCredits', axis=1, inplace=True)
df.drop('TestDesc', axis=1, inplace=True)
#df.drop('TotalBudgetRemaining', axis=1, inplace=True)
#df.drop('TuitionRemaining', axis=1, inplace=True)
#df.drop('Fisap_Income', axis=1, inplace=True)

# Dubious origin state
df.drop('OriginState', axis=1, inplace=True)

In [ ]:
# Look at the columns in the data
df.columns

In [ ]:
df.EFC.describe()

## Part B - Set the option flags for this run

In [ ]:
# Which years are around?
np.unique(df['yr'])

### Set All The Flags

1. Set the year
2. Set the number of interaction terms to use
3. Visualize or not
4. Normalize or not
5. Simple or detailed accuracy metrics

In [ ]:
use_all_years = False
year = 2015

use_pugo_amounts = False

show_viz = False

normalize_metrics = False

detailed_accuracy_metrics = True

f_score_analysis = False

save_results = True

run_tpot = False

stratify_split = True

model_choice = 'boosting' # options are 'boosting' and 'logit'

#### Remove entries with missing data. 

In [ ]:
if use_all_years == False:
    df = df.loc[df['yr'] == year]

df.drop('yr', axis=1, inplace=True)
df = df.reset_index(drop=True)

df = df[np.isfinite(df['EFC'])]

df.isnull().sum()

## Part C - Derived Features

### Part C1 - Create derived features (feature engineering)

* need_after_grants = total budgets minus grants
* need_after_grantsandincome = total budgets minus grants minus efc
* the amount a student would spend after all grants / scholarships

In [ ]:
#df['need_after_grants'] = df['TotalBudget'] - df['UW_Grants'] - df['Grants_and_Scholarships']
#df['need_after_grants_efc'] = df['TotalBudget'] - df['UW_Grants'] - df['Grants_and_Scholarships'] - df['EFC']
if use_pugo_amounts:
    df['tuition_after_grants'] = df['Tuition'] - df['PurpleAndGoldAmount'] - df['Grants_and_Scholarships']
else:
    df['tuition_after_grants'] = df['Tuition'] - df['Grants_and_Scholarships']
#df['tuition_after_grants_efc'] = df['Tuition'] - df['UW_Grants'] - df['Grants_and_Scholarships'] - df['EFC']

df.ix[df.tuition_after_grants < 0, 'tuition_after_grants'] = 0

df.drop('TotalBudget', axis=1, inplace=True)
df.drop('Tuition', axis=1, inplace=True)
df.drop('UW_Grants', axis=1, inplace=True)
df.drop('Grants_and_Scholarships', axis=1, inplace=True)
df.drop('AcademicRank', axis=1, inplace=True)
df.drop('PurpleAndGoldAmount', axis=1, inplace=True)

### Part C2 - Add retention info (feature engineering)

* 5-year graduation rate
* avg quarters to graduate
* avg+1 stdev quarters to graduate
* avg-1 stdev quarters to graduate

In [ ]:
retention_df['5yr_grad_rate'] = retention_df['graduated_student_count'] * 1.0 / retention_df['student_count']

In [ ]:
retention_df['avgplus1_quarters_to_graduate'] = retention_df['avg_quarters_to_graduate'] + retention_df['stdev_quarters_to_graduate']
retention_df['avgminus1_quarters_to_graduate'] = retention_df['avg_quarters_to_graduate'] - retention_df['stdev_quarters_to_graduate']
retention_df.ix[retention_df.avgminus1_quarters_to_graduate < 0, 'avgminus1_quarters_to_graduate'] = 0

In [ ]:
retention_df.drop('graduated_student_count', axis=1, inplace=True)
retention_df.drop('student_count', axis=1, inplace=True)
retention_df.drop('stdev_quarters_to_graduate', axis=1, inplace=True)

In [ ]:
#retention_df[~retention_df.applymap(np.isreal).all(1)]

In [ ]:
#retention_df.first_major_category.unique()

In [ ]:
df['first_major_category'] = 'Unknown'
df.ix[df.BioAndMed == 1, 'first_major_category'] = 'BioAndMed'
df.ix[df.CompSci == 1, 'first_major_category'] = 'CompSci'
df.ix[df.Business == 1, 'first_major_category'] = 'Business'
df.ix[df.Eng == 1, 'first_major_category'] = 'Eng'
df.ix[df.PsychAnthro == 1, 'first_major_category'] = 'PsychAnthro'
df.ix[df.ArtDesign == 1, 'first_major_category'] = 'ArtDesign'
df.ix[df.Law == 1, 'first_major_category'] = 'Com, PoliSci, Law, IntlStudies'
df.ix[df.IntlStudies == 1, 'first_major_category'] = 'Com, PoliSci, Law, IntlStudies'
df.ix[df.Nursing == 1, 'first_major_category'] = 'Nursing'
df.ix[df.Com == 1, 'first_major_category'] = 'Com, PoliSci, Law, IntlStudies'
df.ix[df.Math == 1, 'first_major_category'] = 'MathAndScience'
df.ix[df.PhysicsAstr == 1, 'first_major_category'] = 'MathAndScience'
df.ix[df.Environ == 1, 'first_major_category'] = 'MathAndScience'
df.ix[df.English == 1, 'first_major_category'] = 'Com, PoliSci, Law, IntlStudies'

In [ ]:
df['LowerHighSchoolGPA'] = 0
df.ix[df.HighSchoolGPA > 3.0, 'LowerHighSchoolGPA'] = 3
df.ix[df.HighSchoolGPA > 3.4, 'LowerHighSchoolGPA'] = 3.4
df.ix[df.HighSchoolGPA > 3.6, 'LowerHighSchoolGPA'] = 3.6
df.ix[df.HighSchoolGPA > 3.8, 'LowerHighSchoolGPA'] = 3.8

In [ ]:
df['UpperHighSchoolGPA'] = 3
df.ix[df.HighSchoolGPA > 3, 'UpperHighSchoolGPA'] = 3.4
df.ix[df.HighSchoolGPA > 3.4, 'UpperHighSchoolGPA'] = 3.6
df.ix[df.HighSchoolGPA > 3.6, 'UpperHighSchoolGPA'] = 3.8
df.ix[df.HighSchoolGPA > 3.8, 'UpperHighSchoolGPA'] = 4.0

In [ ]:
#df.first_major_category.unique()

In [ ]:
dfr = df.merge(retention_df, how='left', on=['first_major_category','LowerHighSchoolGPA','UpperHighSchoolGPA'])

In [ ]:
dfr.drop('first_major_category', axis=1, inplace=True)

In [ ]:
list(dfr.columns)

In [ ]:
#dfr['avg_quarters_to_graduate'].describe()

In [ ]:
#dfr[pd.isnull(dfr['avg_quarters_to_graduate'])]

In [ ]:
#dfr.first_major_category.head()

In [ ]:
%matplotlib inline

## Part D - Normalize, clean up, and dummy-code various factors

1. Convert Categorical Features to binary ones (dummy coding)
2. Create more derived features (feature engineering)
3. Normalize everything

### Part D3 - Normalize everything

* http://machinelearningmastery.com/rescaling-data-for-machine-learning-in-python-with-scikit-learn/

**Option Flags**

If 'normalize_metrics' is set, then we normalize all the features using standard scaling. Otherwise we leave them alone.

In [ ]:
finaiddummy_df = pd.get_dummies(dfr['FinAidSegment'], prefix='finaid')
acaddummy_df = pd.get_dummies(dfr['AcademicSegment'], prefix='acad')
eddummy_df = pd.get_dummies(dfr['HighestParentalEducation'], prefix='ed')

eddummy_df['ed_1m'] = eddummy_df['ed_1']+eddummy_df['ed_2']+eddummy_df['ed_3']+eddummy_df['ed_4']+eddummy_df['ed_5']+eddummy_df['ed_6']
eddummy_df['ed_2m'] = eddummy_df['ed_2']+eddummy_df['ed_3']+eddummy_df['ed_4']+eddummy_df['ed_5']+eddummy_df['ed_6']
eddummy_df['ed_3m'] = eddummy_df['ed_3']+eddummy_df['ed_4']+eddummy_df['ed_5']+eddummy_df['ed_6']
eddummy_df['ed_4m'] = eddummy_df['ed_4']+eddummy_df['ed_5']+eddummy_df['ed_6']
eddummy_df['ed_5m'] = eddummy_df['ed_5']+eddummy_df['ed_6']

# Investigate fin aid dummy variables
# finaiddummy_df.head()

# Investigate academic dummy variables
# acaddummy_df.head()

# Join the binary variables to the data
df_1 = dfr.join(eddummy_df)
df_2 = df_1.join(finaiddummy_df)
df_3 = df_2.join(acaddummy_df)

df_3.drop('FinAidSegment', axis=1, inplace=True)
df_3.drop('AcademicSegment', axis=1, inplace=True)
df_3.drop('HighestParentalEducation', axis=1, inplace=True)

df_3 = df_3.reset_index(drop=True)

#df_3.head()

In [ ]:
df_IsEnrolled = df_3.IsEnrolledOrMatriculated.copy()

if detailed_accuracy_metrics:
    print ('df_IsEnrolled has',df_IsEnrolled.count(),'records, and', df_IsEnrolled.sum(),'are enrolled')
    print ('df3 has',df_3.TestScore.count(),' records')

In [ ]:
if normalize_metrics:
    scaler = preprocessing.StandardScaler().fit(df_3)
    # print out the scales so we can use them in the optimization
    print (np.column_stack((df_3.columns, np.transpose(scaler.scale_))))
    #save off the column names
    df_3_columns = df_3.columns 
    df_4 = pd.DataFrame(scaler.transform(df_3))
    df_4.columns = df_3_columns #re-assign the column names
    # Re-add is enrolled or matriculated to convert it back to an int
    df_4.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)
    df_5 = pd.DataFrame(df_IsEnrolled).join(df_4)
    #print 'df_4 has ',df_4.count(),' records'
else: 
    # Use this to skip the scaling step
    df_5 = df_3.copy()

## Part E - Split into training and test sets

1. Come up with interaction terms
2. Use a 70-30 split

### Part E1 - Come up with interaction terms 

**Use PolynomialFeatures**

* Very useful - http://stackoverflow.com/questions/33257199/how-to-generate-many-interaction-terms-in-pandas
* Reference - http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html



* Complicated - http://www.dummies.com/programming/big-data/data-science/data-science-how-to-create-interactions-between-variables-with-python/
* http://patsy.readthedocs.io/en/latest/
* http://stats.idre.ucla.edu/stata/seminars/deciphering-interactions-in-logistic-regression/
* https://www.datarobot.com/blog/multiple-regression-using-statsmodels/
* http://stackoverflow.com/questions/33050104/difference-between-the-interaction-and-term-for-formulas-in-statsmodels-ols

**Option Flags**

if 'full_interaction_terms' is set, then we use *all* interaction terms. Otherwise we limit ourselves to the ones only found useful when running the model in the past.

In [ ]:
# Create the X and Y variables
y = df_5.IsEnrolledOrMatriculated
x = df_5.copy()

x.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)

if detailed_accuracy_metrics:
    print (x.columns)

### Part E2 - Split in a 70-30 split

In [ ]:
if stratify_split == True:
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42, stratify=y)
else:
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)

print ('x_train has',x_train.AcademicScore.count())
print ('y_train has',y_train.count())
print ('')
print ('x_test has',x_test.AcademicScore.count())
print ('y_test has',y_test.count())

df_train = x_train.join(y_train)

#df_train.describe()
#del x, y

## Part F - Find and run classifiers

* http://statsmodels.sourceforge.net/devel/example_formulas.html
* http://statsmodels.sourceforge.net/stable/generated/statsmodels.discrete.discrete_model.Logit.html#statsmodels.discrete.discrete_model.Logit


In [ ]:
# See if there are any columns that won't run in a logit cleanly
if detailed_accuracy_metrics:
    df_train[~df_train.applymap(np.isreal).all(1)]

### Part F1 - Optionally use TPOT to do identify good classifiers

In [ ]:
if run_tpot:
    tpot = TPOTClassifier(generations=4, population_size=15, verbosity=3, scoring='f1')
    tpot.fit(x_train, y_train)
    print(tpot.score(x_test, y_test))
    tpot.export('tpot_enrollment_pipeline.py')

### Part F2 - Run models manually

**Gradient Boosting**

In [ ]:
if model_choice == 'boosting':
    ## gradient boosting
    clf = GradientBoostingClassifier(learning_rate=0.2, n_estimators=500, verbose=1)
    clf.fit(x_train, y_train)
    #
    probs = clf.predict_proba(x_train)
    predicts = clf.predict(x_train)
    #
    probs_test = clf.predict_proba(x_test)
    predicts_test = clf.predict(x_test)

In [ ]:
if model_choice == 'boostingv2':
    #tpot_data = np.recfromcsv('PATH/TO/DATA/FILE', delimiter='COLUMN_SEPARATOR', dtype=np.float64)
    #features = np.delete(tpot_data.view(np.float64).reshape(tpot_data.size, -1), tpot_data.dtype.names.index('class'), axis=1)
    #training_features, testing_features, training_classes, testing_classes = \
    #train_test_split(features, tpot_data['class'], random_state=42)
    #
    clf = make_pipeline(
        make_union(VotingClassifier([("est", GradientBoostingClassifier(learning_rate=0.2, max_features=None, n_estimators=500))]), FunctionTransformer(lambda X: X)),
        #Nystroem(gamma=0.66, kernel="poly", n_components=49),
        Binarizer(threshold=49.0),
        GaussianNB()
    )
    #
    clf.fit(x_train, y_train)
    probs = clf.predict_proba(x_train)
    predicts = clf.predict(x_train)
    #
    probs_test = exported_pipeline.predict_proba(x_test)
    predicts_test = exported_pipeline.predict(x_test)

In [ ]:
#dir(clf)

In [ ]:
#if model_choice == 'boosting':
fi = enumerate(clf.feature_importances_)
cols = x_train.columns
sorted([(value*100, cols[i]) for (i,value) in fi if value > 0.001], reverse=True)

**Logistic** (just for reference)

## Part G - Evaluate the results for accuracy

1. Look at the logit accuracy scores
2. Look at the ROC/AUC scores
3. Look at the initial confusion matrix
4. Optimize for F scores
5. Look at feature importance


**Good Resources**

* https://github.com/justmarkham/DAT4/blob/master/notebooks/08_linear_regression.ipynb
* http://blog.yhat.com/posts/logistic-regression-and-python.html

### Part G1 - look at the accuracy scores lift

In [ ]:
if detailed_accuracy_metrics:
    print ('Baseline training accuracy is', 1.0-y_train.mean())
    # Baseline accuracy is 85.48%
    print ('Training score for',model_choice,'is', clf.score(x_train, y_train))
    # 0.75451707501655474 after class_weight = 'balanced. A bad idea
    # BAD IDEA - 0.88080597862075494 use 'sag' solver. Don't do that
    print ('Improvement above the training baseline is',(clf.score(x_train, y_train)) - (1.0-y_train.mean()))
    print ('Percent improvement above the training baseline is',((clf.score(x_train, y_train)) - (1.0-y_train.mean())) / y_train.mean())

In [ ]:
# Baseline accuracy
print ('Baseline test accuracy is', 1.0-y_test.mean())
# Baseline accuracy is 85.48%

print ('Test score for',model_choice,'is', clf.score(x_test, y_test))

print ('Improvement above the test baseline is',(clf.score(x_test, y_test)) - (1.0-y_test.mean()))
print ('Percent improvement above the test baseline is',((clf.score(x_test, y_test)) - (1.0-y_test.mean())) / y_test.mean())

### Part G2 - Look at the ROC/AUC scores

In [ ]:
if detailed_accuracy_metrics:
    # generate evaluation metrics
    print ('Training Set AUC:',metrics.roc_auc_score(y_train, probs[:, 1]))

In [ ]:
# generate evaluation metrics
print ('Test Set AUC:',metrics.roc_auc_score(y_test, probs_test[:, 1]))

In [ ]:
# Compute the FPR, TPR
false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_test, probs_test[:, 1])
roc_auc = metrics.auc(false_positive_rate, true_positive_rate)

# Plot an ROC curve
plt.title('Receiver Operating Characteristic')
plt.plot(false_positive_rate, true_positive_rate, 'b',
label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

### Part G3 - Look at the initial confusion matrix

In [ ]:
y_array = np.array(y_train)
confusion_output = metrics.confusion_matrix(y_array, predicts)
#print (confusion_output)
true_positive = confusion_output[1][0] # student enrolled and we predicted they would enroll
false_positive = confusion_output[1][1] # student enrolled and we predicted they wouldn't enroll
true_negative = confusion_output[0][0] # student didn't enroll and we predicted they wouldn't enroll
false_negative = confusion_output[0][1] # student didn't enroll and we predicted they would enroll 
precision = true_positive * 1.0 / (true_positive + false_positive)
recall = true_positive * 1.0 / (true_positive + false_negative)
f_score = (precision * recall * 2.0) / (precision + recall)

print ('Of the',true_positive+false_positive,'students who enrolled at the UW, we correctly predicted',true_positive,'of them')
print ('Of the',true_positive+false_negative,'students who we predicted would enroll at the UW',true_positive,'of them did')
if detailed_accuracy_metrics:
    print ('True positive',true_positive,', False Positive',false_positive)
    print ('True negative',true_negative,', False Negative',false_negative)
    print ('Precision:',precision,'Recall:', recall, 'F-Score:',f_score)

In [ ]:
# Now do it for the TEST set
y_array = np.array(y_test)
confusion_output = metrics.confusion_matrix(y_array, predicts_test)
#print (confusion_output)
true_positive = confusion_output[1][0] # student enrolled and we predicted they would enroll
false_positive = confusion_output[1][1] # student enrolled and we predicted they wouldn't enroll
true_negative = confusion_output[0][0] # student didn't enroll and we predicted they wouldn't enroll
false_negative = confusion_output[0][1] # student didn't enroll and we predicted they would enroll 
precision = true_positive * 1.0 / (true_positive + false_positive)
recall = true_positive * 1.0 / (true_positive + false_negative)
f_score = (precision * recall * 2.0) / (precision + recall)

print ('Of the',true_positive+false_positive,'students who enrolled at the UW, we correctly predicted',true_positive,'of them')
print ('Of the',true_positive+false_negative,'students who we predicted would enroll at the UW',true_positive,'of them did')
if detailed_accuracy_metrics:
    print ('True positive',true_positive,', False Positive',false_positive)
    print ('True negative',true_negative,', False Negative',false_negative)
    print ('Precision:',precision,'Recall:', recall, 'F-Score:',f_score)

#### Results Tracking

* Store in a Google sheet, https://docs.google.com/a/uw.edu/spreadsheets/d/1ne3dhmafyKZQNlTm7dlONYXvflGP-r3K-CeHfnOO6Qw/edit?usp=sharing 


## Part H - Propensity Score Matching

### Resources

**Highly Useful Code and Guides**

* https://stats.stackexchange.com/questions/206832/matched-pairs-in-python-propensity-score-matching
* https://www.quora.com/Where-can-I-find-a-good-writeup-about-propensity-models


**Code**
* http://nbviewer.jupyter.org/github/kellieotto/StatMoments/blob/master/PSM.ipynb
* https://github.com/laurencium/causalinference
* https://github.com/MacHu-GWU/ctmatching-project

**Documentation/Guides**
* http://laurence-wong.com/software/


### Steps
* Predict the probabilities
* Split into buckets
* Visualize the causal factor of each bucket

In [ ]:
# Predict the probabilities
predictions = clf.predict_proba(x)

#sns.jointplot(x="tuition_after_grants", y="IsEnrolledOrMatriculated", data=df);

In [ ]:
# Visualize the probabilities
sns.distplot(predictions[:, 1], hist=True, rug=False);

In [ ]:
predict_df = df_3[['Fisap_Income', 'EFC', 'AcademicScore','DeltaAcademicMinusIncome', 'LowerIncomePct', 'DeltaParentEdMinusAcademic','IsEnrolledOrMatriculated']].copy(deep=True)
predict_df['PropensityScore'] = predictions[:, 1]

In [ ]:
# Split into buckets
bins = [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3,0.4,0.6, 0.8, 1.0]
bin_name = [0,1,2,3,4,5,6,7,8,9]

In [ ]:
predict_df['PropensityBin'] = pd.cut(predict_df['PropensityScore'], bins, labels=bin_name)

#### Use the CausalInference library
* https://github.com/laurencium/causalinference/blob/master/docs/tex/vignette.pdf

In [ ]:
#causal = CausalModel(Y, D, X)
# Y 'an N-vector of observed outcomes'
# D 'an N-vector of treatment status indicators'
# X 'an N-by-K matrix of covariates'

In [ ]:
d_causal = predict_df.IsEnrolledOrMatriculated
y_causal = predict_df.PropensityScore

x_causal = predict_df.copy()
x_causal.drop('PropensityScore', axis=1, inplace=True)
#x_causal.drop('PropensityBin', axis=1, inplace=True)
x_causal.drop('IsEnrolledOrMatriculated', axis=1, inplace=True)

print (x_causal.columns)

In [ ]:
x_causal2 = x_causal.apply(pd.to_numeric, errors='ignore')

In [ ]:
[x_causal2.iloc[:,i].apply(type).value_counts() for i in range(x_causal2.shape[1])]

In [ ]:
causal = CausalModel(y_causal, d_causal, x_causal2.as_matrix(), )

In [ ]:
# Summary stats
print(causal.summary_stats)

In [ ]:
# Propensity scoring
causal.est_propensity_s()
print (causal.propensity)

In [ ]:
causal.propensity.keys()

In [ ]:
sys.setrecursionlimit(1500)

In [ ]:
causal.est_via_matching()
causal.est_via_ols()
causal.est_via_weighting()
#causal.est_via_blocking()
print(causal.estimates)

In [ ]:
dir (causal.estimates)